In [1]:
import os
import boto3
import pandas as pd

DB_ACCESS_KEY = pd.read_csv('DB_ACCESS_KEY.csv')
aws_access_key_id = DB_ACCESS_KEY['Access key ID'][0]
aws_secret_access_key = DB_ACCESS_KEY['Secret access key'][0]
rds_key_pass = pd.read_csv('db_key_pass.csv')
username = rds_key_pass['ID'][0]
password = rds_key_pass['PASS'][0]
url = rds_key_pass['URL'][0]

In [3]:
from boto3.dynamodb.conditions import Key

def getPastStockPrices(refresh_counter) -> pd.DataFrame:
    """
    returns a pandas dataframe structured as follows:
    company name, ticker, sentiment score, sentiment magnitude, sentiment score change, sentiment magnitude change
    """
    # specify key and secret key
    aws_access_key_id = os.environ['DB_ACCESS_KEY']
    aws_secret_access_key = os.environ['DB_SECRET_KEY']
    # # create a boto3 client and import all stock prices from it
    dynamodb = boto3.resource('dynamodb', region_name='us-east-2', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
    table = dynamodb.Table('StockPrices')
    # keep scanning until we have all the data in the table
    # keep scanning until we have all the data in the table
    response = table.scan()
    data = response['Items']
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        data.extend(response['Items'])
    # convert the data to a pandas dataframe and drop the stock column
    stock_prices = pd.DataFrame(data).drop(columns=['Stock'], errors='ignore')
    stock_prices.set_index('Date', inplace=True)
    # convert Date column to datetime
    stock_prices.index = pd.to_datetime(stock_prices.index)
    # make the index the Date column
    stock_prices.sort_index(ascending=False, inplace=True)
    return stock_prices


In [ ]:
import pymysql
from sqlalchemy import create_engine

# Establish connection to the MySQL database
conn = pymysql.connect(host=url, user=username, password=password, db='stock_data')

# Create a SQLAlchemy engine object
engine = create_engine(f'mysql+pymysql://{username}:{password}@{url}/stock_data', echo=False)

# Convert the pandas DataFrame to a MySQL table
sentiment_ticker_list.to_sql(name='Sentiments', con=engine, if_exists='append', index=False)

# Close the connection
conn.close()